In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pickle

In [19]:
violations = pd.read_csv('data/violations.csv')
permits = pd.read_csv('data/permits.csv')
complaints = pd.read_csv('data/detroit-311.csv')
crimes = pd.read_csv('data/detroit-crime.csv')

/home/ishaan/ishaan/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ishaan/ishaan/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
permits['BLD_PERMIT_TYPE'].value_counts()

DISM         5859
Dismantle    1274
Name: BLD_PERMIT_TYPE, dtype: int64

In [22]:
buildings = pd.read_csv('data/buildings.csv')

In [67]:
def distance(pt,c):
    pt = pt*np.pi/180
    c = c*np.pi/180
    lat,lng = c.T[0],c.T[1]
    deltay = lat - pt[0]
    deltax = (lng - pt[1])*np.cos((lat+pt[0])/2)
    R = 6371e3
    d = R*np.sqrt(np.power(deltax,2)+np.power(deltay,2))
    return d.min(), np.argmin(d)

def check_building(row,sites):
    lat,long = row['LAT'],row['LONG']
    dis, _ = distance(np.array([lat,long]),sites)
    if(dis<=100):
        return 'blighted'
    else:
        return 'not_blighted'
    

In [68]:
sites = np.array(permits.dropna(axis=0,subset=['LAT','LONG'])[['LAT','LONG']])

In [69]:
buildings['label'] = buildings.apply(lambda row: check_building(row,sites) ,axis=1)

In [72]:
buildings['label'].value_counts()

not_blighted    9357
blighted        3923
Name: label, dtype: int64

In [74]:
buildings.reset_index(inplace=True)

In [80]:
buildings = buildings.rename(columns = {'index':'building_id'})

In [82]:
buildings.to_csv('data/train_labels.csv',index=None)